## Cheapoair.com autosuggestion list extraction

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import xlsxwriter
from time import sleep
from string import ascii_lowercase
import itertools
import time


start1 = time.time()
path  = r'C:\Program Files (x86)\Google\chromedriver.exe'
driver = webdriver.Chrome(r'C:\Program Files (x86)\Google\chromedriver.exe')
driver.get("https://www.cheapoair.com/hotels/")
driver.find_element_by_xpath('/html/body/form/div/div/div[1]/div[1]/section/div[1]/div[2]/div/div/div[2]/div/div/div[2]').click()


def scrap(input_):

    
    link = driver.find_element_by_xpath('//*[@id="from"]')
    link.send_keys(input_)
    sleep(1.5)
    link1 = driver.find_element_by_xpath("//ul[@class='suggestion-box__list autoSuggest__list']").text
    new = link1.split("\n")
    del new[0]
    
    
    lst = []
    for i in new:
        lst.append(i.split(','))
        
    my_columns = ['character','Place','State', 'Country']
    df = pd.DataFrame(columns = my_columns)

    for i in range(len(lst)):
        if len(lst[i]) == 2:
            lst[i].insert(0,"N/A")
        lst[i].insert(0,input_)

    temp = []
    for i in range(len(lst)):
        df = df.append(pd.Series([lst[i][0],lst[i][1],lst[i][2],lst[i][3]],index = my_columns),ignore_index = True)
    
    index = pd.Index(range(1, len(lst)+1, 1))
    df = df.set_index(index)
    link.send_keys('\b')
    return df 
                 
my_columns = ['character','Place','State', 'Country']
final_df = pd.DataFrame(columns = my_columns)


def iter_all_strings():
    for size in itertools.count(1):
        for s in itertools.product(ascii_lowercase, repeat=size):
            yield "".join(s)
                 
for i in iter_all_strings():
    df1 = scrap(i)
    final_df = pd.concat([final_df, df1])
    if i =='zzzz':
        break
        
writer = pd.ExcelWriter('last_sample.xlsx', engine= 'xlsxwriter')
final_df.to_excel(writer,'last_sample', index = True)
writer.save()
end = time.time()
print("time taken:  ",end- start1,"sec")